In [1]:
# Web scraping test
! pip install BeautifulSoup4 s3fs requests tqdm

In [0]:
# import libraries
import os
import re
import json
import s3fs
from datetime import datetime
import tqdm
import urllib
import requests
from bs4 import BeautifulSoup

In [0]:
# Parse data based on the structure of the page
# We are going to create a dictionary that we will then write to a json file
# Each dictionary key will be the name of the image

# Note the following constructs: 

data_dict = {}
base_url = 'https://www.zappos.com/women-boots/CK_XARCz1wHAAQHiAgMBAhg.zso'
n = 1

while True:
  url = base_url + "?p={}".format(n)
  page = urllib.request.urlopen(url)
  soup = BeautifulSoup(page, 'html.parser')
  all_links = soup.find_all('a')
  link_count = len(all_links)
  l = 0
  # Process data
  for link in all_links:
    aria_label = link.get('aria-label')
    if aria_label and link.img:
      # Sort to make sure that what we get back has a price, ensuring it is a boot we want to include
      if aria_label.find('$') != -1:
        l += 1
        key = link.img['src'].split('/')[-1] # Use the image name as the key
        rating = re.search("Rated (.*?)\.", aria_label)
        if rating:
          rating = rating.group(1)

        data_dict[key] = {
            'shoe_name': aria_label.split('. By')[0],
            'brand': re.search("By (.*?) \$", aria_label).group(1).strip('.'),
            'price': "$" + re.search("\$(.*?) ", aria_label).group(1).strip('.'),
            'style': re.search("Style: (.*?)\.", aria_label).group(1),
            'rating': rating, 
            'image_source': link.img['src']
        }
  if l == 0:
    break

  n += 1


In [0]:
# Connect to s3 and upload the things

s3 = s3fs.S3FileSystem(anon=False)
bucket = "skafos.bootfinder/"
dataset = datetime.now().strftime("%Y%m%d%H%M%S")
img_path = '/boot_images/'
meta = 'boots_meta_data.json'

In [0]:
# write the meta data dictionary
with s3.open(bucket + dataset + '/' + meta, 'w') as f:
  f.write(json.dumps(data_dict))

In [0]:
# collect and write images
for k, v in data_dict.items():
  with s3.open(bucket + dataset + img_path + k, 'wb') as f:
    f.write(requests.get(v['image_source']).content)